In [9]:
# This is simply the beginning of pyirc.py that sets up relevant parameters
# and reads in configuration settings;  example uses 'example_config_vis'
import sys
sys.path.append('../')
import time
import re
import numpy
import pyirc
import matplotlib
import matplotlib.pyplot as plt

class EmptyClass:
  pass

outstem = 'default_output'
use_cmap = 'gnuplot'

mydet = ''
lightfiles = []
darkfiles = []
vislightfiles = []
visdarkfiles = []
formatpars = 1
nx = 32
ny = 32
tslices = [3,11,13,21]
tslicesM2a = []
tslicesM2b = []
tslicesM3 = []
fullref = True
sensitivity_spread_cut = .1
critfrac = 0.75
mychar = 'Basic'
hotpix = False
ref_for_hotpix_is_autocorr = False
hotpix_logtspace = False
hotpix_slidemed = False

# order parameters
s_bfe = 2     # order of BFE parameters
p_order = 0   # non-linearity polynomial table coefficients (table at end goes through order p_order)
              # set to zero to turn this off

# Parameters for basic characterization
basicpar = EmptyClass()
basicpar.epsilon = .01
basicpar.subtr_corr = True
basicpar.noise_corr = True
basicpar.reset_frame = 1
basicpar.subtr_href = True
basicpar.full_corr = True
basicpar.leadtrailSub = False
basicpar.g_ptile = 75.
basicpar.fullnl = False
basicpar.use_allorder = False

# Parameters for BFE
bfepar = EmptyClass()
bfepar.epsilon = .01
bfepar.treset = basicpar.reset_frame
bfepar.blsub = True
bfepar.fullnl = False

# Plotting parameters
narrowfig = False

# Read in information
config_file = '../example_config_vis'
with open(config_file) as myf: content = myf.read().splitlines()
is_in_light = is_in_dark = is_in_vislight = is_in_visdark = False
maskX = [] # list of regions to mask
maskY = []
for line in content:
  # Cancellations
  m = re.search(r'^[A-Z]+\:', line)
  if m: is_in_light = is_in_dark = is_in_vislight = is_in_visdark = False

  # Searches for files -- must be first given the structure of this script!
  # The visible flats and darks must come after IR flats and darks
  if is_in_light:
    m = re.search(r'^\s*(\S.*)$', line)
    if m: lightfiles += [m.group(1)]
  if is_in_dark:
    m = re.search(r'^\s*(\S.*)$', line)
    if m: darkfiles += [m.group(1)]
  if is_in_vislight:
    m = re.search(r'^\s*(\S.*)$', line)
    if m: vislightfiles += [m.group(1)]
  if is_in_visdark:
    m = re.search(r'^\s*(\S.*)$', line)
    if m: visdarkfiles += [m.group(1)]
        
  # -- Keywords go below here --

  # Search for outputs
  m = re.search(r'^OUTPUT\:\s*(\S*)', line)
  if m: outstem = m.group(1)
  # Search for input files
  m = re.search(r'^LIGHT\:', line)
  if m: is_in_light = True
  m = re.search(r'^DARK\:', line)
  if m: is_in_dark = True
  m = re.search(r'^VISLIGHT\:', line)
  if m: is_in_vislight = True
  m = re.search(r'^VISDARK\:', line)
  if m: is_in_visdark = True

  # Format
  m = re.search(r'^FORMAT:\s*(\d+)', line)
  if m: formatpars = int(m.group(1))

  # Bin sizes
  m = re.search(r'^NBIN:\s*(\d+)\s+(\d+)', line)
  if m:
    nx = int(m.group(1))
    ny = int(m.group(2))

  # Characterization type (Basic or Advanced)
  m = re.search(r'^CHAR:\s*(\S+)', line)
  if m:
     mychar = m.group(1)
     if mychar.lower()=='advanced':
       m = re.search(r'^CHAR:\s*(\S+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\S+)', line)
       if m:
         tchar1 = int(m.group(2))
         tchar2 = int(m.group(3))
         ncycle = int(m.group(4))
         ipnltype = m.group(5)
       else:
         print ('Error: insufficient arguments: ' + line + '\n')
         exit()

  # Time slices
  m = re.search(r'^TIME:\s*(\d+)\s+(\d+)\s+(\d+)\s+(\d+)', line)
  if m: tslices = [ int(m.group(x)) for x in range(1,5)]
  m = re.search(r'^TIME2A:\s*(\d+)\s+(\d+)\s+(\d+)\s+(\d+)', line)
  if m: tslicesM2a = [ int(m.group(x)) for x in range(1,5)]
  m = re.search(r'^TIME2B:\s*(\d+)\s+(\d+)\s+(\d+)\s+(\d+)', line)
  if m: tslicesM2b = [ int(m.group(x)) for x in range(1,5)]
  m = re.search(r'^TIME3:\s*(\d+)\s+(\d+)\s+(\d+)\s+(\d+)', line)
  if m: tslicesM3 = [ int(m.group(x)) for x in range(1,5)]
  #
  # reference time slice
  m = re.search(r'^TIMEREF:\s*(\d+)', line)
  if m: bfepar.treset = basicpar.reset_frame = int(m.group(1))

  # reference pixel subtraction
  m = re.search(r'^REF\s+OFF', line)
  if m: fullref = False

  # sensitivity spread cut
  m = re.search(r'^SPREAD:\s*(\S+)', line)
  if m: sensitivity_spread_cut = float(m.group(1))

  # variance parameters
  m = re.search(r'^QUANTILE:\s*(\S+)', line)
  if m: basicpar.g_ptile = float(m.group(1))
  # correlation parameters
  m = re.search(r'^EPSILON:\s*(\S+)', line)
  if m: bfepar.epsilon = basicpar.epsilon = float(m.group(1))
  m = re.search(r'^IPCSUB:\s*(\S+)', line)
  if m: basicpar.leadtrailSub = m.group(1).lower() in ['true', 'yes']

  # Other parameters
  m = re.search(r'^DETECTOR:\s*(\S+)', line)
  if m: mydet = m.group(1)
  m = re.search(r'^COLOR:\s*(\S+)', line)
  if m: use_cmap = m.group(1)

  # Classical non-linearity
  m = re.search(r'^NLPOLY:\s*(\S+)\s+(\S+)\s+(\S+)', line)
  if m:
    p_order = int(m.group(1))
    nlfit_ts = int(m.group(2))
    nlfit_te = int(m.group(3))

  m = re.search(r'^FULLNL:\s*(\S+)\s+(\S+)\s+(\S+)', line)
  if m:
    basicpar.fullnl = m.group(1).lower() in ['true', 'yes']
    bfepar.fullnl = m.group(2).lower() in ['true', 'yes']
    basicpar.use_allorder = m.group(3).lower() in ['true', 'yes']

  # Hot pixels
  # (adu min, adu max, cut stability, cut isolation)
  m = re.search(r'^HOTPIX:\s*(\S+)\s+(\S+)\s+(\S+)\s+(\S+)', line)
  if m:
    hotpix = True
    hotpix_ADU_range = [ float(m.group(x)) for x in range(1,5)]
  #
  # change reference for hot pixels from last point to autocorr
  m = re.search(r'^HOTREF\s+AUTOCORR', line)
  if m: ref_for_hotpix_is_autocorr = True
  # log spacing for times?
  m = re.search(r'^HOTPIX\s+LOGTSPACE', line)
  if m: hotpix_logtspace = True
  # sliding median alpha method?
  m = re.search(r'^HOTPIX\s+SLIDEMED', line)
  if m: hotpix_slidemed = True

  # Mask regions by hand
  m = re.search(r'^MASK:\s*(\d+)\s+(\d+)', line)
  if m:
    maskX = maskX + [int(m.group(1))]
    maskY = maskY + [int(m.group(2))]

  # Control figures
  m = re.search(r'^NARROWFIG', line)
  if m: narrowfig = True

In [10]:
# set up array size parameters
pyirc.swi.addbfe(s_bfe)
pyirc.swi.addhnl(p_order)
print ('Number of output field per superpixel =', pyirc.swi.N)

# Check number of slices available
NTMAX = 16384
for f in lightfiles+darkfiles:
  nt = pyirc.get_num_slices(formatpars, f)
  if nt<NTMAX: NTMAX=nt

# Copy basicpar parameters to bfebar
bfepar.use_allorder = basicpar.use_allorder

print ('Output will be directed to {:s}*'.format(outstem))
print ('Light files:', lightfiles)
print ('Dark files:', darkfiles)
print ('Visible light files:', vislightfiles)
print ('"Visible" dark files:', visdarkfiles)
print ('Time slices:', tslices, 'max=',NTMAX)
print ('Mask regions:', maskX, maskY)
# 
if len(lightfiles)!=len(darkfiles) or len(lightfiles)<2:
  print ('Failed: {:d} light files and {:d} dark files'.format(len(lightfiles), len(darkfiles)))
  exit()

Number of output field per superpixel = 61
Output will be directed to out/this_det12*
Light files: ['/fs/scratch/PCON0003/cond0007/SCA20829-qy/20191016_95K_1p1m0p1_q_yield_1400nm_gr3_filt4_20829_001.fits', '/fs/scratch/PCON0003/cond0007/SCA20829-qy/20191016_95K_1p1m0p1_q_yield_1400nm_gr3_filt4_20829_002.fits']
Dark files: ['/fs/scratch/PCON0003/cond0007/SCA20829/20191018_95K_1p1m0p1_ch0_1400nm_gr3_filt5_shutter_closed_20829_001.fits', '/fs/scratch/PCON0003/cond0007/SCA20829/20191018_95K_1p1m0p1_ch0_1400nm_gr3_filt5_shutter_closed_20829_002.fits']
Visible light files: ['/fs/scratch/PCON0003/cond0007/SCA20829-qy/20191016_95K_1p1m0p1_q_yield_480nm_gr3_filt6_20829_001.fits', '/fs/scratch/PCON0003/cond0007/SCA20829-qy/20191016_95K_1p1m0p1_q_yield_480nm_gr3_filt6_20829_002.fits']
"Visible" dark files: ['/fs/scratch/PCON0003/cond0007/SCA20829/20191018_95K_1p1m0p1_ch0_1400nm_gr3_filt5_shutter_closed_20829_001.fits', '/fs/scratch/PCON0003/cond0007/SCA20829/20191018_95K_1p1m0p1_ch0_1400nm_gr3_fi

In [11]:
# De-bugging pyirc stuff;  doesn't need to be run anymore
# gives an idea of how the 
def get_nside(formatpars):
  if formatpars==1: return 4096
  if formatpars==2: return 2048
  if formatpars==3: return 4096
  if formatpars==4: return 4096

import fitsio
filename='/fs/scratch/PCON0003/cond0007/SCA20829-qy/20191016_95K_1p1m0p1_q_yield_1400nm_gr3_filt4_20829_001.fits'
formatpars=4
xyrange=[0, 4, 0, 128]
tslices=[1, 3, 4, 6]
# Recommended True (False defaults to astropy tools, which work but are slow because of the way this script works)
use_fitsio = True

# Get dimensions of output cube
nxuse = xyrange[1]-xyrange[0]
nyuse = xyrange[3]-xyrange[2]
ntslice_use = len(tslices)
output_cube = numpy.zeros((ntslice_use, nyuse, nxuse))
print(ntslice_use, nyuse, nxuse)
fileh = fitsio.FITS(filename)
# Is there no time slice 13??
N = get_nside(formatpars)
for ts in range(ntslice_use):
    t = tslices[ts]
    #print(t)
    output_cube[ts,:,:] = numpy.array(fileh[1][0, t-1, xyrange[2]:xyrange[3], xyrange[0]:xyrange[1]])
fileh.close()


4 128 4


In [12]:
# Additional parameters
# Size of a block
N = pyirc.get_nside(formatpars)
# Side lengths
dx = N//nx
dy = N//ny
# Pixels in a block
npix = dx*dy

# Make table of reference pixel corrections for Method 1
# This is only happening now on the visible files
if fullref:
  lightref = pyirc.ref_array(vislightfiles, formatpars, ny, tslices, False)
  darkref = pyirc.ref_array(vislightfiles, formatpars, ny, tslices, False)
else:
  lightref = numpy.zeros((len(vislightfiles), ny, 2*len(tslices)+1))
  darkref = numpy.zeros((len(visdarkfiles), ny, 2*len(tslices)+1))
basicpar.subtr_href = fullref

# more allocations
my_dim = pyirc.swi.N
full_info = numpy.zeros((ny,nx,my_dim))
is_good = numpy.zeros((ny,nx))

if p_order>0:
  # now coefficients for the info table
  # note that in 'abs' mode, the full_info[:,:,0] grid is not actually used, it
  #   is just there for consistency of the format
  # I moved this up here since we want to have these coefficients before the main program runs
  nlcubeX, nlfitX, nlderX, pcoefX = pyirc.gen_nl_cube(
          vislightfiles, formatpars, [basicpar.reset_frame, nlfit_ts, nlfit_te], [ny,nx],
        full_info[:,:,0], 'abs', False)
  # fill in
  for iy in range(ny):
    for ix in range(nx):
      if pcoefX[1,iy,ix]!=0:
        full_info[iy,ix,pyirc.swi.Nbb] = -pcoefX[0,iy,ix]/pcoefX[1,iy,ix]
        for o in range(2,pyirc.swi.p+1):
          full_info[iy,ix,pyirc.swi.Nbb+o-1] = pcoefX[o,iy,ix]/pcoefX[1,iy,ix]**o
      else:
        full_info[iy,ix,pyirc.swi.Nbb] = -1e49 # error code

# Detector characterization data in a cube (basic characterization + BFE Method 1)
# Stdout calls are a progress indicator
#

In [15]:
# This is the function that will take the correlation function bits out of pyirc.basic
# It is called corr_5x5 in pyirc.py
def corr(region_cube, dark_cube, tslices, lightref, darkref, ctrl_pars, verbose):
  # Extract basic parameters
  num_files = region_cube.shape[0]-1
  nt = region_cube.shape[1]
  dy = region_cube.shape[2]
  dx = region_cube.shape[3]
  npix = dx*dy
  print(num_files)
  # Get means and variances at the early and last slices
  # region cube is 4D array of dimension number of files +1, number tslices, ymax-ymin, xmax-xmin
  box1 = region_cube[0:num_files,0,:,:] - region_cube[0:num_files,1,:,:]
  box2 = region_cube[0:num_files,0,:,:] - region_cube[0:num_files,-1,:,:]
  box2Noise = dark_cube[0:num_files,0,:,:] - dark_cube[0:num_files,-1,:,:]
    
  # Correlations of neighboring pixels, in DN^2
  tCH = tCV = tCD = 0
  epsilon=0.01
  corr_mask = region_cube[-1,0,:,:]
  print(corr_mask.shape)
  for if1 in range(1,num_files):
    for if2 in range(if1):
      print("if1,if2",if1,if2)
      temp_box = box2[if1,:,:] - box2[if2,:,:]
      print("temp_box",temp_box.shape)
      nrun = 1 # need to change to allow for noise
      for icorr in range(nrun):
        # clipping
        cmin = pyirc.pyIRC_percentile(temp_box,corr_mask,100*epsilon)
        cmax = pyirc.pyIRC_percentile(temp_box,corr_mask,100*(1-epsilon))
        this_mask = numpy.where(numpy.logical_and(temp_box>cmin,temp_box<cmax),\
1,0) * corr_mask

        if numpy.sum(this_mask)<1: return [] # FAIL!
        # mean subtraction
        #mean_of_temp_box = numpy.sum(temp_box*this_mask)/numpy.sum(this_mask)
        #if subtr_corr and newMeanSubMethod: temp_box -= mean_of_temp_box  # figure out corrections later

        # Correlations in horizontal and vertical directions
        maskCV = numpy.sum(this_mask[:-1,:]*this_mask[1:,:])
        #print("this_mask",this_mask)

        maskCH = numpy.sum(this_mask[:,:-1]*this_mask[:,1:])
        maskCV2 = numpy.sum(this_mask[:-2,:]*this_mask[2:,:])
        maskCH2 = numpy.sum(this_mask[:,:-2]*this_mask[:,2:])
 
        CV = numpy.sum(this_mask[:-1,:]*this_mask[1:,:]*temp_box[:-1,:]*temp_box[1:,:])
        #print("CV",CV)
        CH = numpy.sum(this_mask[:,:-1]*this_mask[:,1:]*temp_box[:,:-1]*temp_box[:,1:])
        if maskCH<1 or maskCV<1: return []
        CH /= maskCH
        CV /= maskCV
        # Need to do all the diagonal calculations and 
        temp_box = box2Noise[if1,:,:] - box2Noise[if2,:,:]
        # Then normalize since we're double-counting
        print("CV",CV)
  return CH, CV

In [16]:
# Comparing different ways of measuring correlations
# (note: something not quite right about the below, needs a check...)
import numpy as np
## Example 5x5 array (but in reality we'll work with much larger arrays)
x = np.arange(25).reshape((5,5))
print("Example array:", x)

### We can first measure the vertical correlations by brute force
### The first thing to check is correlations separated by two pixels
cvx = np.zeros_like(x)
for jdx in range(x.shape[1]):
    for idx in range(x.shape[0]):
        if (jdx<2):
            cvx[jdx][idx]=x[jdx,idx]*x[jdx+2,idx]
        elif (jdx>=2)&(jdx<x.shape[1]-2):
            cvx[jdx][idx]=x[jdx,idx]*x[jdx+2,idx]+x[jdx-2,idx]*x[jdx,idx]
        else:
            cvx[jdx][idx]=x[jdx-2,idx]*x[jdx,idx]
#print(cvx)
print("Brute force correlations:",np.sum(cvx)/2.)
print("Compare to what is called like in pyirc.basic:", np.sum(x[:-2,:]*x[2:,:]))

### Same exercise can be done for the horizontal correlations
### Also separated by two pixels
# This is currently written differently than above
chx = np.zeros_like(x)
for jdx in range(x.shape[1]):
    for idx in range(x.shape[0]-2):
        chx[jdx][idx]=x[jdx,idx]*x[jdx,idx+2]
#print(chx)
print(np.sum(chx))
print("Compare to:", np.sum(x[:,:-2]*x[:,2:]))

Example array: [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
Brute force correlations: 2065.0
Compare to what is called like in pyirc.basic: 2065
2905
Compare to: 2905


In [17]:
numpy.set_printoptions(threshold=sys.maxsize)
# Tests, only run for single iy, ix for example? i.e. single super-pixel
#print ('Method 1, progress of calculation:')
#sys.stdout.write('|')
for iy in range(ny): sys.stdout.write(' ')
#print ('| <- 100%')
#sys.stdout.write('|')
#for iy in range(ny):
for i in range(1):
  sys.stdout.write('*'); sys.stdout.flush()
  for ix in range(1):
  #for ix in range(nx):
    region_cube = pyirc.pixel_data(vislightfiles, formatpars, [dx*ix, dx*(ix+1), dy*iy, dy*(iy+1)], tslices,
                  [sensitivity_spread_cut, True], False)
    dark_cube = pyirc.pixel_data(visdarkfiles, formatpars, [dx*ix, dx*(ix+1), dy*iy, dy*(iy+1)], tslices,
                  [sensitivity_spread_cut, False], False)
    
    info = corr(region_cube, dark_cube, tslices, lightref[:,iy,:], darkref[:,iy,:], basicpar, False)
    #info = pyirc.basic(region_cube, dark_cube, tslices, lightref[:,iy,:], darkref[:,iy,:], basicpar, False)
    print(region_cube.shape)
    #print(info[0])
    #print(info[1])
    #exit()

#print ('|')


                                                                                                                                *2
(32, 32)
if1,if2 1 0
temp_box (32, 32)
CV 50.24209078404402
(3, 4, 32, 32)


In [18]:
"""
# Visualization of what's in region_cube
import matplotlib.pyplot as plt
%matplotlib inline
#print(region_cube.shape)
plt.imshow(region_cube[0,0,:,:], origin='lower left')
print(region_cube[0,0,:,:].min(), region_cube[0,0,:,:].max())
print(region_cube[1,0,:,:].min(), region_cube[1,0,:,:].max())
plt.show()
"""

"\nimport matplotlib.pyplot as plt\n%matplotlib inline\n#print(region_cube.shape)\nplt.imshow(region_cube[0,0,:,:], origin='lower left')\nprint(region_cube[0,0,:,:].min(), region_cube[0,0,:,:].max())\nprint(region_cube[1,0,:,:].min(), region_cube[1,0,:,:].max())\nplt.show()\n"

In [29]:
# Tests with convolve2d
from scipy.signal import convolve2d, correlate2d
# If we define an x_flip[j,i]=f[Ny-1-j,Nx-1-i]
# then convolve2d between x and x_flip we should get the correlation
# Looks like correlated2d maybe does this with the flip built in
corr2d_answer=correlate2d(x,x)

Ny=Nx=5
x_flip = np.flip(x)
print(x_flip)

[[24 23 22 21 20]
 [19 18 17 16 15]
 [14 13 12 11 10]
 [ 9  8  7  6  5]
 [ 4  3  2  1  0]]


In [30]:
# Checks that numpy flip is producing what we want
print(x_flip[0,0])
print(x[4,4])
print(x_flip[0,2])
print(x[4,2])

24
24
22
22


In [31]:
conv2d_answer=convolve2d(x,x_flip)
print('conv2d:',conv2d_answer)
print('corr2d:',corr2d_answer)

conv2d: [[   0   24   71  140  230  220  191  144   80]
 [ 120  278  472  700  960  820  652  458  240]
 [ 335  712 1128 1580 2065 1700 1308  892  455]
 [ 620 1276 1964 2680 3420 2760 2084 1396  700]
 [ 950 1920 2905 3900 4900 3900 2905 1920  950]
 [ 700 1396 2084 2760 3420 2680 1964 1276  620]
 [ 455  892 1308 1700 2065 1580 1128  712  335]
 [ 240  458  652  820  960  700  472  278  120]
 [  80  144  191  220  230  140   71   24    0]]
corr2d: [[   0   24   71  140  230  220  191  144   80]
 [ 120  278  472  700  960  820  652  458  240]
 [ 335  712 1128 1580 2065 1700 1308  892  455]
 [ 620 1276 1964 2680 3420 2760 2084 1396  700]
 [ 950 1920 2905 3900 4900 3900 2905 1920  950]
 [ 700 1396 2084 2760 3420 2680 1964 1276  620]
 [ 455  892 1308 1700 2065 1580 1128  712  335]
 [ 240  458  652  820  960  700  472  278  120]
 [  80  144  191  220  230  140   71   24    0]]


In [28]:
# We see that the correlations as defined in pyirc.basic produces the same vertical and horizontal
# correlations 
print("Compare to what is called like in pyirc.basic for CV:", np.sum(x[:-1,:]*x[1:,:]))
print("Compare to what is called like in pyirc.basic for CH:", np.sum(x[:,:-1]*x[:,1:]))

Compare to what is called like in pyirc.basic: 3420
Compare to what is called like in pyirc.basic: 3900
